Code Description:

This codes slices and dices (filters) the Customer´s Database (AWCustomers.csv) 50 times, once for each row in the
AWTest-classification file. On each iteration, The AWCsutomers dataframe is sliced with the value of each feature
of the row in turn.

In [111]:
import pandas as pd
import numpy as np
import os
import math

In [112]:
# Best Answer So Far
Answer = [1,0,0,1,1,1,0,1,0,1,0,0,1,1,1,1,1,1,1,0,1,0,1,1,0,0,1,
          1,1,1,0,1,1,1,0,1,1,0,1,1,1,0,1,0,0,1,1,0,1,1]

In [113]:
# Customer ID to use for classification challenge
Customer = [11219,11241,11352,11904,11908,12113,12221,12286,12292,12305,12807,12820,12892,13310,13349,
              13482,13640,13838,14320,14327,14380,14477,14564,14652,14698,14778,15085,15177,15388,15464,
              15573,15626,15727,15738,15866,15875,16184,16394,16575,17536,17605,17723,17726,18171,18345,
              18382,18842,18956,19019,19315]

In [114]:
#['CustomerID','Education','Occupation','Gender','MaritalStatus','NumberCarsOwned', 'NumberChildrenAtHome','TotalChildren',
# 'HomeOwnerFlag','Age','IncomeNorm', 'BikeBuyer', 'AvgMonthSpend' ]

In [115]:
# Read Customers Database
path = (r'C:\Home\Ser\MOOCs\MSDataProg\Project\AWCustomers\Datasets')
filename = 'CustomersDatabase002.csv'
pathfile= os.path.join(path,filename)  
db = pd.read_csv(pathfile, sep=',', index_col=0)
db.reset_index(inplace=True, drop=True) # CustomerID is removed here

# Group into [0,1,>1] instead of [0,1,2,3,4,5]
db['NumberCarsOwned'].replace([2,3,4,5],'>1',inplace=True)
# Group into [0,>0] instead of [0,1,2,3]
db['NumberChildrenAtHome'].replace([1,2,3],'>0',inplace=True)
# Group into [0,12,3] instead of [0,1,2,3]
db['TotalChildren'].replace([1,2],12, inplace=True)

dropcols = ['AvgMonthSpend',
            'HomeOwnerFlag', 
            #'NumberCarsOwned', 
            'TotalChildren',
            #'NumberChildrenAtHome',
           ]
db.drop(dropcols, axis=1, inplace=True)

In [116]:
# Read  dataset to estimate classification (AWTest-classification)
path = (r'C:\Home\Ser\MOOCs\MSDataProg\Project\AWCustomers\Datasets')
filename = 'CustomersTest002.csv'  
pathfile= os.path.join(path,filename)  
dfclass = pd.read_csv(pathfile, sep=',', index_col=0)
dfclass = dfclass.ix[Customer]
dfclass.reset_index(inplace=True,drop=True) # to remove Customer ID

# Group into [0,1,>1] instead of [0,1,2,3,4,5]
dfclass['NumberCarsOwned'].replace([2,3,4,5],'>1',inplace=True)
# Group into [0,>0] instead of [0,1,2,3]
dfclass['NumberChildrenAtHome'].replace([1,2,3],'>0',inplace=True)
# Group into [0,12,3] instead of [0,1,2,3]
dfclass['TotalChildren'].replace([1,2],12, inplace=True)


dropcols = ['TotalChildren',
            'HomeOwnerFlag', 
            #'NumberCarsOwned', 
            #'NumberChildrenAtHome',
            ]
dfclass.drop(dropcols, axis=1, inplace=True)
dfclass

,Education,Occupation,Gender,MaritalStatus,NumberCarsOwned,NumberChildrenAtHome,Age,IncomeNorm
0,High School,Skilled Manual,M,M,>1,>0,46,5300
1,Partial College,Skilled Manual,F,S,0,0,39,10755
2,Partial College,Clerical,M,M,>1,0,56,7413
3,Graduate Degree,Professional,F,S,>1,0,27,3551
4,Graduate Degree,Professional,M,S,>1,0,30,8915
5,High School,Clerical,M,M,>1,>0,45,2668
6,Graduate Degree,Clerical,F,M,>1,0,36,2034
7,Graduate Degree,Professional,M,S,>1,0,27,10732
8,Bachelors,Skilled Manual,M,M,1,0,52,9159
9,Partial College,Clerical,M,M,0,>0,42,3535


In [117]:
Zero = []
One  = []
Flag = []
headers = dfclass.columns
for i in range(50):
    df=db
    for col in headers:
        value=dfclass.loc[i,col]
        if col=='Age':
            age_limit = 2
            age_lower = value - age_limit
            age_upper = value + age_limit
            mask= (df[col]>= age_lower) & (df[col]<= age_upper)
        elif col=='IncomeNorm':
            Income_limit = 1000 # 2K income window
            Income_lower = value - Income_limit
            Income_upper = value + Income_limit
            mask= (df[col]>= Income_lower) & (df[col]<= Income_upper)          
        else:    
            mask = df[col]==value  
            
        # Slice df with mask defined above    
        df= df[mask]
        
    # Count bikebuyers within the remaining dataframe slice    
    dfrows = len(df.index)  
    n_one = df['BikeBuyer'].sum()
    n_zero = dfrows - n_one
    diff = n_one - n_zero
    if diff > 0:
        flag = 1
    elif diff<0:
        flag = 0
    else:
        flag = None
    One.append(n_one)
    Zero.append(n_zero)
    Flag.append(flag)

In [118]:
# Print Results

dic = {'One'  :One,
       'Zero' :Zero,
       'Flag' :Flag,
       'Best' :Answer}
df = pd.DataFrame(dic, index=Customer)
df['Diff'] = abs(df['Best'] - df['Flag'])
print df['Diff'].sum()
df

3.0


,Best,Flag,One,Zero,Diff
11219,1,1.0,7,0,0.0
11241,0,NaN,0,0,NaN
11352,0,NaN,0,0,NaN
11904,1,1.0,6,4,0.0
11908,1,1.0,7,2,0.0
12113,1,1.0,5,0,0.0
12221,0,NaN,0,0,NaN
12286,1,1.0,6,2,0.0
12292,0,1.0,1,0,1.0
12305,1,1.0,1,0,0.0


In [119]:
dfclass['CustomerID'] = Customer
dfclass

,Education,Occupation,Gender,MaritalStatus,NumberCarsOwned,NumberChildrenAtHome,Age,IncomeNorm,CustomerID
0,High School,Skilled Manual,M,M,>1,>0,46,5300,11219
1,Partial College,Skilled Manual,F,S,0,0,39,10755,11241
2,Partial College,Clerical,M,M,>1,0,56,7413,11352
3,Graduate Degree,Professional,F,S,>1,0,27,3551,11904
4,Graduate Degree,Professional,M,S,>1,0,30,8915,11908
5,High School,Clerical,M,M,>1,>0,45,2668,12113
6,Graduate Degree,Clerical,F,M,>1,0,36,2034,12221
7,Graduate Degree,Professional,M,S,>1,0,27,10732,12286
8,Bachelors,Skilled Manual,M,M,1,0,52,9159,12292
9,Partial College,Clerical,M,M,0,>0,42,3535,12305


        elif col=='NumberCarsOwned':
            if value == 0:
                mask= (df[col]==0)
            elif value ==1:
                mask= (df[col]==1)
            else: 
                mask= (df[col]> 1)
        elif col=='TotalChildren':
            if value == 0:
                mask= (df[col]==0)  
            else: 
                mask= (df[col]> 0)                          
        elif col=='NumberChildrenAtHome':
            if value == 0:
                mask= (df[col]==0)  
            else: 
                mask= (df[col]> 0)  